<a href="https://colab.research.google.com/github/jjlee6496/one-by-one/blob/main/GAN%EA%B5%AC%ED%98%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Torch

## GAN

In [ ]:
import os
import torch
import torchvision
import torch.nn as nn
from torchvision import transforms
from torchvision.utils import save_image

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
latent_size = 64
hidden_size = 256
image_size = 784
num_epochs = 200
batch_size = 100
sample_dir = 'samples'

In [ ]:
if not os.path.exists(sample_dir):
  os.makedirs(sample_dir)

In [ ]:
transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.5],
                                     std=[0.5])])

In [ ]:
mnist = torchvision.datasets.MNIST(root='../../data/',
                                   train = True,
                                   transform=transform,
                                   download=True)
data_loader = torch.utils.data.DataLoader(dataset=mnist,
                                          batch_size=batch_size,
                                          shuffle=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../../data/MNIST/raw/train-images-idx3-ubyte.gz to ../../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../../data/MNIST/raw



In [ ]:
from torch.nn.modules.activation import LeakyReLU
D = nn.Sequential(
    nn.Linear(image_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size,1 ),
    nn.Sigmoid()
)

In [ ]:
G = nn.Sequential(
    nn.Linear(latent_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, image_size),
    nn.Tanh()
)

In [ ]:
D = D.to(device)
G = G.to(device)

In [ ]:
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0002)
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0002)

In [ ]:
def denorm(x):
  out = (x + 1) / 2
  return out.clamp(0, 1)

def reset_grad():
  d_optimizer.zero_grad()
  g_optimizer.zero_grad()

In [ ]:
total_step = len(data_loader)
for epoch in range(num_epochs):
  for i, (images, _) in enumerate(data_loader):
    images = images.reshape(batch_size, -1).to(device)

    real_labels = torch.ones(batch_size, 1).to(device)
    fake_labels = torch.zeros(batch_size, 1).to(device)
    ###################################
    # Train discriminator
    outputs = D(images)
    d_loss_real = criterion(outputs, real_labels)
    real_score = outputs

    z = torch.randn(batch_size, latent_size).to(device)
    fake_images = G(z)
    outputs = D(fake_images)
    d_loss_fake = criterion(outputs, fake_labels)
    fake_score = outputs

    d_loss = d_loss_real + d_loss_fake
    reset_grad()
    d_loss.backward()
    d_optimizer.step()

    ###############################
    #Train Generator
    z = torch.randn(batch_size, latent_size).to(device)
    fake_images = G(z)
    outputs = D(fake_images)

    g_loss = criterion(outputs, real_labels)

    reset_grad()
    g_loss.backward()
    g_optimizer.step()

    if (i+1) % 200 == 0:
      print('Epoch [{}/{}], Step[{}/{}], d_loss: {:.4f}, g_loss:{:.4f}, D(x): {:.2f}, D(G(x)): {:.2f}'
            .format(epoch, num_epochs, i+1, total_step, d_loss.item(), g_loss.item(),
                    real_score.mean().item(), fake_score.mean().item()))
  if (epoch+1) == 1:
    images = images.reshape(images.size(0), 1, 28, 28)
    save_image(denorm(images), os.path.join(sample_dir, 'real_images.png'))

  fake_images = fake_images.reshape(fake_images.size(0), 1, 28, 28)
  save_image(denorm(fake_images), os.path.join(sample_dir, 'fake_images-{}.png'.format(epoch+1)))

torch.save(G.state_dict(), 'G.ckpt')
torch.save(D.state_dict(), 'D.ckpt')

Epoch [0/200], Step[200/600], d_loss: 0.0379, g_loss:4.1542, D(x): 1.00, D(G(x)): 0.03
Epoch [0/200], Step[400/600], d_loss: 0.0722, g_loss:5.0975, D(x): 0.98, D(G(x)): 0.05
Epoch [0/200], Step[600/600], d_loss: 0.1160, g_loss:3.8131, D(x): 0.95, D(G(x)): 0.06
Epoch [1/200], Step[200/600], d_loss: 0.0301, g_loss:6.8186, D(x): 0.99, D(G(x)): 0.02
Epoch [1/200], Step[400/600], d_loss: 0.3293, g_loss:2.9552, D(x): 0.86, D(G(x)): 0.11
Epoch [1/200], Step[600/600], d_loss: 0.1179, g_loss:5.6663, D(x): 0.95, D(G(x)): 0.04
Epoch [2/200], Step[200/600], d_loss: 0.3065, g_loss:3.1970, D(x): 0.90, D(G(x)): 0.10
Epoch [2/200], Step[400/600], d_loss: 0.2263, g_loss:3.6846, D(x): 0.94, D(G(x)): 0.10
Epoch [2/200], Step[600/600], d_loss: 0.2371, g_loss:5.5576, D(x): 0.93, D(G(x)): 0.09
Epoch [3/200], Step[200/600], d_loss: 0.6798, g_loss:2.8280, D(x): 0.81, D(G(x)): 0.20
Epoch [3/200], Step[400/600], d_loss: 0.3725, g_loss:2.7473, D(x): 0.88, D(G(x)): 0.16
Epoch [3/200], Step[600/600], d_loss: 2.102

In [ ]:
torch.save(G.state_dict(), 'G.ckpt')
torch.save(D.state_dict(), 'D.ckpt')

In [ ]:
from PIL import Image
from IPython.display import Image as Img
import natsort

In [ ]:
# Save Images to gif file
path = 'samples'
img_list = os.listdir(path)
img_list = [path + '/' + x for x in img_list]
img_list = natsort.natsorted(img_list)
img_list

['samples/fake_images-1.png',
 'samples/fake_images-2.png',
 'samples/fake_images-3.png',
 'samples/fake_images-4.png',
 'samples/fake_images-5.png',
 'samples/fake_images-6.png',
 'samples/fake_images-7.png',
 'samples/fake_images-8.png',
 'samples/fake_images-9.png',
 'samples/fake_images-10.png',
 'samples/fake_images-11.png',
 'samples/fake_images-12.png',
 'samples/fake_images-13.png',
 'samples/fake_images-14.png',
 'samples/fake_images-15.png',
 'samples/fake_images-16.png',
 'samples/fake_images-17.png',
 'samples/fake_images-18.png',
 'samples/fake_images-19.png',
 'samples/fake_images-20.png',
 'samples/fake_images-21.png',
 'samples/fake_images-22.png',
 'samples/fake_images-23.png',
 'samples/fake_images-24.png',
 'samples/fake_images-25.png',
 'samples/fake_images-26.png',
 'samples/fake_images-27.png',
 'samples/fake_images-28.png',
 'samples/fake_images-29.png',
 'samples/fake_images-30.png',
 'samples/fake_images-31.png',
 'samples/fake_images-32.png',
 'samples/fake_im

In [ ]:
imgs = [Image.open(x) for x in img_list[::10]]
im = imgs[0]
im.save('out.gif', save_all=True, append_images=imgs[1:],loop=0xff, duration= 500)

In [81]:
imgs2 = [Image.open(x) for x in img_list[::5]]
im2 = imgs[0]
im2.save('out2.gif', save_all=True, append_images=imgs2[1:],loop=0xff, duration= 200)